<a href="https://colab.research.google.com/github/lisah2u/DatasetScraper/blob/master/Deep_research_agent_with_o3_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apify-client pydantic openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os
from getpass import getpass
from typing import List, Dict, Any, Optional, Type
from pydantic import BaseModel, Field
from openai import OpenAI
from datetime import datetime, timedelta
import json
from apify_client import ApifyClient
import requests

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass()

··········


In [ ]:
CHAT_MODEL_NAME = 'gpt-4o-mini'
client = OpenAI()

In [ ]:
os.environ['SPIDER_API_KEY'] = getpass()

··········


In [ ]:
os.environ['APIFY_API_KEY'] = getpass()

In [ ]:
os.environ['SERPER_API_KEY'] = getpass()

··········


In [ ]:
os.environ['RAPID_API_KEY'] = getpass()

In [ ]:
def run_reasoning_agent(user_input: str, tool_map):
    messages = [{"role": "developer", "content": "You are an internet researcher, you always use tools to find latest internet data based on research topic and give detailed & conprehensive result; Never make up information"},{"role": "user", "content": user_input}]

    while True:
        response = client.chat.completions.create(
            model="o3-mini",
            reasoning_effort="high",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )

        response_message = response.choices[0].message

        messages.append(response_message)

        if not response_message.tool_calls:
            return response_message.content

        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f'**[INFO] Calling tool: {function_name} with args {function_args}**')
            function_call = tool_map[function_name]
            function_response = function_call(**function_args)

            print(f'**[FUNCTION CALL RESULT] {function_name}: {function_response}**')

            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": json.dumps(function_response)
            })

In [ ]:
def run_4o_agent(user_input: str, tool_map):
    messages = [{"role": "developer", "content": "You are an internet researcher, you always use tools to find latest internet data based on research topic and give detailed & conprehensive result; Never make up information"},{"role": "user", "content": user_input}]

    while True:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )

        response_message = response.choices[0].message

        messages.append(response_message)

        if not response_message.tool_calls:
            return response_message.content

        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f'**[INFO] Calling tool: {function_name} with args {function_args}**')
            function_call = tool_map[function_name]
            function_response = function_call(**function_args)

            print(f'**[FUNCTION CALL RESULT] {function_name}: {function_response}**')

            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": json.dumps(function_response)
            })

In [ ]:
# Define functions:
# - scrape website
# - google search
# - extract linkedin profile
# - extract instagram


def scrape_website(url: str) -> str:
    headers = {
        'Authorization': f'Bearer {os.getenv("SPIDER_API_KEY")}',
        'Content-Type': 'application/json',
    }

    payload = {
        "limit": 10,
        "return_format": "markdown",
        "url": url
    }

    response = requests.post('https://api.spider.cloud/crawl', headers=headers, json=payload)
    response.raise_for_status()
    return response.json()

# def google_search(query: str) -> str:
#     headers = {
#         'X-API-KEY': os.getenv("SERPER_API_KEY"),
#         'Content-Type': 'application/json',
#     }

#     payload = {
#         "q": query,
#         "num_results": 20,  # if supported
#         "start": 0          # to set the offset for pagination
#     }

#     response = requests.post('https://google.serper.dev/search', headers=headers, json=payload)
#     response.raise_for_status()
#     return response.text

def google_search(query: str) -> str:
    """
    Performs a Google search and returns the results as a string.

    This version of the function accepts only the query as an argument,
    to match the expected usage within run_reasoning_agent based on the error.
    """
    headers = {
        'X-API-KEY': os.getenv("SERPER_API_KEY"),
        'Content-Type': 'application/json',
    }

    payload = {
        "q": query
    }

    response = requests.post('https://google.serper.dev/search', headers=headers, json=payload)
    response.raise_for_status()
    return response.text


def extract_linkedin_profile(linkedin_url: str) -> str:
    headers = {
        "x-rapidapi-key": os.getenv("RAPID_API_KEY"),
        "x-rapidapi-host": "fresh-linkedin-profile-data.p.rapidapi.com",
    }

    params = {
        "linkedin_url": linkedin_url,
        "include_skills": "false",
        "include_certifications": "false",
        "include_publications": "false",
        "include_honors": "false",
        "include_volunteers": "false",
        "include_projects": "false",
        "include_patents": "false",
        "include_courses": "false",
        "include_organizations": "true",
        "include_profile_status": "false",
        "include_company_public_url": "false",
    }

    response = requests.get('https://fresh-linkedin-profile-data.p.rapidapi.com/get-linkedin-profile', headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def extract_instagram(instagram_url: str) -> str:
    headers = {
        'Authorization': f'Bearer {os.getenv("APIFY_API_KEY")}',
        'Content-Type': 'application/json',
    }

    payload = {
        "directUrls": [instagram_url],
        "resultsType": "posts",
        "resultsLimit": 5,
        "searchType": "hashtag",
        "searchLimit": 1,
        "addParentData": False,
    }

    response = requests.post('https://api.apify.com/v2/actor-runs/shu8hvrXbJbY3Eb9W/run-sync-get-dataset-items', headers=headers, json=payload)
    response.raise_for_status()
    return response.json()

## Define tools

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "scrape_website",
      "description": "Scrapes a website and returns the content in markdown format",
      "strict": True,
      "parameters": {
        "type": "object",
        "required": [
          "url"
        ],
        "properties": {
          "url": {
            "type": "string",
            "description": "The URL of the website to scrape"
          }
        },
        "additionalProperties": False
      }
    }
  },
  {
    "type": "function",
    "function": {
        "name": "google_search",
        "description": "Performs a Google search and returns the results as a string",
        "strict": True,
        "parameters": {
          "type": "object",
          "required": [
            "query"
          ],
          "properties": {
            "query": {
              "type": "string",
              "description": "The search query string"
            }
          },
          "additionalProperties": False
        }
    }
  },
  {
    "type": "function",
    "function": {
        "name": "extract_linkedin_profile",
        "description": "Extracts LinkedIn profile information from a given LinkedIn URL.",
        "strict": True,
        "parameters": {
          "type": "object",
          "required": [
            "linkedin_url"
          ],
          "properties": {
            "linkedin_url": {
              "type": "string",
              "description": "The LinkedIn profile URL to extract information from, e.g. https://www.linkedin.com/in/cjfollini/."
            }
          },
          "additionalProperties": False
        }
    }
  },
  {
    "type": "function",
    "function": {
        "name": "extract_instagram",
        "description": "Extracts data from Instagram using a given URL",
        "strict": True,
        "parameters": {
          "type": "object",
          "required": [
            "instagram_url"
          ],
          "properties": {
            "instagram_url": {
              "type": "string",
              "description": "The URL of the Instagram page to extract data from, e.g. https://www.instagram.com/saunaamalfi/"
            }
          },
          "additionalProperties": False
        }
      }
    }

]

In [ ]:
tool_map = {
    'scrape_website': scrape_website,
    'google_search': google_search
#    'extract_linkedin_profile': extract_linkedin_profile,
#    'extract_instagram': extract_instagram
}

# Compare results

In [ ]:
query = "I’m researching species that became invasive after people who kept them as pets released them. There’s a certain species of fish that was popularized as a pet by being the main character of the movie Finding Nemo. According to the USGS, where was this fish found as a nonnative species, before the year 2020? I need the answer formatted as the five-digit zip codes of the places the species was found, separated by commas if there is more than one place."
response = run_4o_agent(query, tool_map)
print(f"--FINAL RESULT: {response}")

In [ ]:
query = "I’m researching species that became invasive after people who kept them as pets released them. There’s a certain species of fish that was popularized as a pet by being the main character of the movie Finding Nemo. According to the USGS, where was this fish found as a nonnative species, before the year 2020? I need the answer formatted as the five-digit zip codes of the places the species was found, separated by commas if there is more than one place."
response = run_reasoning_agent(query, tool_map)
print(f"--FINAL RESULT: {response}")

# Test att


In [ ]:
query="Research detailed customer opinions of AT&T’s customer guarantee program, and investigate what analysts may say about this program, both skeptic and bullish views. What is the benefit to at&t and will competitors match it? Include url links to ALL opinions given, and provide final result as an verbose editorial (use tables when appropriate) in markdown format"
response = run_4o_agent(query, tool_map)
print(f"--FINAL RESULT: {response}")

**[INFO] Calling tool: google_search with args {'query': 'AT&T customer guarantee program reviews'}**
**[FUNCTION CALL RESULT] google_search: {"searchParameters":{"q":"AT&T customer guarantee program reviews","type":"search","engine":"google"},"organic":[{"title":"Is New Customer Guarantee a Reason To Choose AT&T?","link":"https://www.highspeedinternet.com/resources/att-offers-guarantee","snippet":"The guarantees are best in class, but may not be a reason to switch to AT&T if you already like your mobile or internet service. That's because ...","date":"Jan 9, 2025","attributes":{"Missing":"program | Show results with:program"},"position":1},{"title":"AT&T Guarentee : r/ATT - Reddit","link":"https://www.reddit.com/r/ATT/comments/1hwjifq/att_guarentee/","snippet":"If it would have “Guaranteed” Taxes and Fees included with the plan price for straightforward pricing this would have been a win. I rolled my ...","date":"Jan 8, 2025","attributes":{"Missing":"reviews | Show results with:review

In [ ]:
query="Research detailed customer opinions of AT&T’s customer guarantee program, and investigate what analysts may say about this program, both skeptic and bullish views. What is the benefit to at&t and will competitors match it? Include url links to ALL opinions given, and provide final result as an verbose editorial (use tables when appropriate) in markdown format"
response = run_reasoning_agent(query, tool_map)
print(f"--FINAL RESULT: {response}")

**[INFO] Calling tool: google_search with args {'query': 'AT&T customer guarantee program customer opinions analyst views benefit competitors match skeptical bullish AT&T customer guarantee program'}**
**[FUNCTION CALL RESULT] google_search: {"searchParameters":{"q":"AT&T customer guarantee program customer opinions analyst views benefit competitors match skeptical bullish AT&T customer guarantee program","type":"search","engine":"google"},"organic":[{"title":"AT&T Unveils First & Only Customer Promise for Wireless & Fiber","link":"https://about.att.com/story/2025/guarantee.html","snippet":"The AT&T Guarantee promises connectivity customers can depend on, deals they want, and the prompt, friendly service they deserve.","date":"Jan 8, 2025","attributes":{"Missing":"opinions analyst views benefit skeptical bullish"},"position":1},{"title":"AT&T Guarantee: Reliable Connectivity & Service","link":"https://www.att.com/why-att/guarantee/","snippet":"AT&T is the first and only carrier that of

### Test 2

In [ ]:
query = "In Nature journal's Scientific Reports conference proceedings from 2012, in the article that did not mention plasmons or plasmonics, what nano-compound is studied? Don't use the prefix nano in your answer if there is one."
response = run_4o_agent(query, tool_map)
print(f"--FINAL RESULT: {response}")

In [ ]:
query = "In Nature journal's Scientific Reports conference proceedings from 2012, in the article that did not mention plasmons or plasmonics, what nano-compound is studied? Don't use the prefix nano in your answer if there is one."
response = run_reasoning_agent(query, tool_map)
print(f"--FINAL RESULT: {response}")

In [ ]:
query="""You are an expert lawyer who researches case law. You are presented with the following backstory of a case:

The Block 188 project, a mixed-use development in Austin, Texas, has been plagued by delays and disputes.
Austin Proper JV, LLC (plantiff), the project owner, has expressed dissatisfaction with Austin Commercial’s performance.
The project, initially scheduled for completion in late 2018, has faced numerous challenges including design changes,
subcontractor performance issues, and coordination problems.
Austin Commercial (defendant) is now facing potential liquidated damages of $4.625 million due to these delays.
The situation is further complicated by incomplete design documents and ongoing changes, which
Austin Commercial claims have contributed significantly to the project’s timeline issues.

Research all relevant Texas case law and legal precedents relating to the dispute and each issue involved with the dispute and describe how each case or legal precedent would govern the dispute.
Identify the top eight cases with similar fact patterns or legal issues to the dispute between the parties, and limit your search to cases that (in order of precedence) are (a) from the jurisdiction associated with the contractual choice of law, or (b) from the jurisdiction where the contract was to be performed;

For each of these cases, analyze the rulings and reasoning and describe how they would apply to the dispute between the parties;

For each of these cases, identify and describe why each case would be favorable or unfavorable to each party. List them by case name and url link with output format as markdown"""
response = run_4o_agent(query, tool_map)
print(f"--FINAL RESULT: {response}")

**[INFO] Calling tool: google_search with args {'query': 'Texas case law liquidated damages design changes construction delays Austin'}**
**[FUNCTION CALL RESULT] google_search: {"searchParameters":{"q":"Texas case law liquidated damages design changes construction delays Austin","type":"search","engine":"google"},"organic":[{"title":"Loggins Const. Co. v. Stephen F. Austin State University Bd. of ...","link":"https://casetext.com/case/loggins-const-co-v-stephen-f-austin-state-university-bd-of-regents","snippet":"Austin State University Board of Regents, appellee (the University), to recover money withheld by the University as liquidated damages on a construction ...","position":1},{"title":"Texas Construction Law Blog - Porter Hedges LLP","link":"https://www.porterhedges.com/texas-construction-law/tips-on-drafting-enforceable-and-effective-liquidated-damages-provisions-under-texas-law","snippet":"In construction contracts, liquidated damages are commonly used for: (1) delays to comple